## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# read in the cleaned csv file from online site (data stored on private server to provide stable static hosting)
df = pd.read_csv("http://www.andrewlane.us/data/crime_data2020-2024.csv") # File is 268MB, allow time for download
df.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,190326475,03/01/2020 12:00:00 AM,03/01/2020 12:00:00 AM,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,AA,Adult Arrest,510.0,998.0,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506
1,200106753,02/09/2020 12:00:00 AM,02/08/2020 12:00:00 AM,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,998.0,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628
2,200320258,11/11/2020 12:00:00 AM,11/04/2020 12:00:00 AM,1700,3,Southwest,356,1,480,BIKE - STOLEN,...,IC,Invest Cont,480.0,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002
3,200907217,05/10/2023 12:00:00 AM,03/10/2020 12:00:00 AM,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,IC,Invest Cont,343.0,NaN,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387
4,200412582,09/09/2020 12:00:00 AM,09/09/2020 12:00:00 AM,630,4,Hollenbeck,413,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,200 E AVENUE 28,NaN,34.0820,-118.2130


In [2]:
# Isolate data that would be available durring a live 911 call and Victom Age (Vict Age) to use for training the model
df = df[['TIME OCC', 'LAT', 'LON', 'Vict Age']]
df.head()

,TIME OCC,LAT,LON,Vict Age
0,2130,34.0375,-118.3506,0
1,1800,34.0444,-118.2628,47
2,1700,34.0210,-118.3002,19
3,2037,34.1576,-118.4387,19
4,630,34.0820,-118.2130,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005104 entries, 0 to 1005103
Data columns (total 4 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   TIME OCC  1005104 non-null  int64  
 1   LAT       1005104 non-null  float64
 2   LON       1005104 non-null  float64
 3   Vict Age  1005104 non-null  int64  
dtypes: float64(2), int64(2)
memory usage: 30.7 MB


In [4]:
df.nunique()

TIME OCC    1439
LAT         5426
LON         4982
Vict Age     104
dtype: int64

In [5]:
# Remove rows with Vict Age < 0 (entry errors)
df = df[df['Vict Age'] > 0]

# create column Is Minor to use to train model
df['Is Minor'] = df['Vict Age'].apply(lambda x: 1 if x < 18 else 0)

# drop the Vict Age column to keep it out of the training data
df = df.drop(columns=['Vict Age'])

In [6]:
df.head()

,TIME OCC,LAT,LON,Is Minor
1,1800,34.0444,-118.2628,0
2,1700,34.0210,-118.3002,0
3,2037,34.1576,-118.4387,0
11,1430,34.0881,-118.1877,0
19,1615,34.1016,-118.3370,0


In [7]:
df.describe()

,TIME OCC,LAT,LON,Is Minor
count,735641.000000,735641.000000,735641.000000,735641.000000
mean,1326.719866,33.997381,-118.085022,0.034801
std,651.051015,1.636055,5.670705,0.183275
min,1.000000,0.000000,-118.667600,0.000000
25%,900.000000,34.015500,-118.433500,0.000000
50%,1400.000000,34.059200,-118.326500,0.000000
75%,1845.000000,34.166400,-118.276000,0.000000
max,2359.000000,34.334300,0.000000,1.000000


In [8]:
# Split our preprocessed data into our features and target arrays
y = df['Is Minor'].values
X = df.drop('Is Minor', axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=8)

In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [10]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 4 # neural units tried: 2,4,8,16,32,64,128,256
# hidden_nodes_layer2 = 2 # multiple layers attempted
# hidden_nodes_layer3 = 2
# hidden_nodes_layer4 = 2

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1,
                          input_dim=number_input_features,
                          activation="relu")
)

# Additional hidden layers
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2,
#                              activation="relu"))
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3,
#                              activation="relu"))
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4,
#                              activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1,
                             activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 16        
                                                                 
 dense_1 (Dense)             (None, 1)                 5         
                                                                 
Total params: 21 (84.00 Byte)
Trainable params: 21 (84.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [12]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=2) # no improved accuracy after 2 epochs

Epoch 1/2
17242/17242 [==============================] - 5s 289us/step - loss: 0.1568 - accuracy: 0.9653
Epoch 2/2
17242/17242 [==============================] - 5s 278us/step - loss: 0.1499 - accuracy: 0.9653


In [13]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

5748/5748 - 1s - loss: 0.1512 - accuracy: 0.9649 - 1s/epoch - 227us/step
Loss: 0.15122394263744354, Accuracy: 0.9649178385734558


In [14]:
# Export our model to HDF5 file
nn.save('victim_is_minor.h5')

/opt/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
